<a href="https://colab.research.google.com/github/xtbtds/ml-zoomcamp/blob/main/lesson7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Bento ML makes it easy to create and package your ML service for production.

# 7.2 Building Your Prediction Service with BentoML
- Create a directory and install bentoml:  
`pip install bentoml`  
- Then save the model that you've created earlier (here we will use xgboost model):
```
import bentoml
bento.xgboost.save_model("credit_risk_model", model)
```
You'll see **tag** and **path** where the model is saved.
- Create a service:




In [ ]:
import bentoml
from bentoml.io import JSON

model_ref = bentoml.xgboost.get("credit_risk_model:oyfd25cj3oijqdu5")

model_runner = model_ref.to_runner()    #scale the model separately from the rest of the service

svc = bentoml.Service("credit_risk_classifier", runners=[model_runner])

@svc.api(input=JSON(), output=JSON())
def classify(application_data):
    prediction = model_runner.predict.run(application_data)
    return { "status": "APPROVED" }

- Run service:  
`bentoml serve service.py:svc`
- Follow provided local link and you'll see automatically generated **Swagger UI**
- Paste some data and execute - you'll see an error Not Supported Type. That's because we've given the model clear data, without transforming by DictVectorizer. Change model saving in step 2:
```
bento.xgboost.save_model("credit_risk_model", model,
                          custom_objects={
                              "dictVectorizer": dv
                          })
```
- Then edit `service.py`:

In [ ]:
...  
dv = model_ref.custom_objects['dictVectorizer']  
...

@svc.api(input=JSON(), output=JSON())
def classify(application_data):
    vector = dv.transform(vector)
...

- To start server automatically after making changes:  
`bentoml serve service.py:svc --reload`
- Edit the model's answer depending on its prediction:

In [ ]:
# in function Classify:
print(prediction)
result = prediction[0]

if result > 0.5:
    return {
        "status": "DECLINED"
    }
elif result > 0.25:
    return {
        "status": "MAYBE"
    }
else:
    return {
        "status": "APPROVED"
    }